# 自然语言处理 NLP

自然语言是人类思维的载体，但其非结构化、歧义性、多样性给机器理解带来巨大挑战，如：

词与义的割裂：同一单词在不同语境中含义不同（如“Apple”指水果或公司）。

形态复杂性：中文无空格分隔、德语复合词、英语时态变化等。

长程依赖：句子语义可能依赖远处词汇（如“I **is** the 9th letter on the vocabulary list”）。

解决路径：将语言转化为机器可处理的数值形式，而**分词 Tokenization**是这一过程的第一步。

# 语言编码与分词基础

1. 语言编码的三层抽象

|层级|描述|示例|
|----|----|----|
|字符级|处理单个字母/符号|"A" → 65 (ASCII)|
|子词级|平衡语义与效率（主流方案）|"unhappy" → ["un","happy"]|
|词级|整词为单位（简单但数量极大）|"cat" → 3047 (词表ID)|

2. 分词器（Tokenizer）的核心作用
   
    文本 → 令牌（Token）序列：将句子拆分为模型可处理的单元
    
    "I love NLP" → ["I", "love", "NLP"]
    
    解决未登录词（OOV）问题：通过子词拆分覆盖生僻词

    "pneumonoultramicroscopicsilicovolcanoconiosis" → ["pneumono", "ultra", "micro", "scopic", "silico", "volcano", "coniosis"]

# 分词器的设计思路

1. 基于规则的分词

    依赖预定义规则，如空格、标点切分。

2. 基于词频统计的分词

    如 Byte Pair Encoding (BPE)（GPT/BERT所用方案）
    
    其核心思想是初始将文本拆分为单字符，然后基于频次等算法等策略合并为多字符加入词典。

    直到词典规模达到预设值。

    *如果基于频次合并则是 BPE 如果基于 频次/子词频次乘积则是 WordPiece，每种策略各有优点和不足。*

4. 基于深度学习的分词

    使用BiLSTM、Transformer等模型

    将分词视为序列标注问题

下面会实现使用 BPE 算法训练一个词汇表，训练文本使用 sklearn.datasets.fetch_20newsgroups，分词前先进行了去除空白字符和按空格预分词。

In [1]:
# 这里会实现一个最简单的语言编解码器，字典只有字母,空格还有测试字符对 'kar'

class Tokenizer:
    def __init__(self, data: list[str]) -> None:
        self.ind2char = ["[UNK]", "[PAD]", "[BOS]", "[EOS]"]
        self.ind2char.extend(data)
        self.single_token_length = sorted(set(len(c) for c in self.ind2char), reverse=True)
        self.char2ind = {c: i for i, c in enumerate(self.ind2char)}

    def encode(self, text: str) -> list[int]:
        tokens: list[int] = []
        while len(text) > 0:
            for length in self.single_token_length:
                if length > len(text):
                    continue
                if token := self.char2ind.get(text[:length]):
                    tokens.append(token)
                    text = text[length:]
                    break
            else:
                text = text[length:]
                tokens.append(self.char2ind["[UNK]"])
        return tokens

    def decode(self, indices: list[int], hyphen: str = "") -> str:
        return hyphen.join(self.ind2char[i] for i in indices)


tokenizer = Tokenizer(list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ") + ["kar"])
print(tokenizer.ind2char)
encoded = tokenizer.encode("Hello karis!!!")
print(encoded)
print(tokenizer.decode(encoded, hyphen="_"))

['[UNK]', '[PAD]', '[BOS]', '[EOS]', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ', 'kar']
[37, 8, 15, 15, 18, 56, 57, 12, 22, 0, 0, 0]
H_e_l_l_o_ _kar_i_s_[UNK]_[UNK]_[UNK]


# BPE 算法

BPE 算法的目标是通过迭代合并高频符号对，逐步构建一个子词词汇表。其核心思想是：

初始化：将词汇表初始化为所有字符。

合并：找到频率最高的符号对，将其合并为一个新的子词，并更新词汇表。

迭代：重复上述过程，直到词汇表达到预设大小或没有更多符号对可合并。

In [2]:
from sklearn.datasets import fetch_20newsgroups
import re


def pre_tokenizers(train_data: list[str]):
    whitespace = re.compile(r"\s+")
    new_train_data = []
    for text in train_data:
        new_train_data.extend(whitespace.sub(" ", text).split())
    return new_train_data


train_data = pre_tokenizers(fetch_20newsgroups(subset="train")["data"][:100])

In [3]:
from collections import Counter

def train_bpe(texts, vocab_size):
    # 初始化词汇表为字符,统计初始符号频率
    symbols_list = [list(text) for text in texts]
    vocab = set()
    for symbols in symbols_list:
        vocab.update(symbols)
    len_vocab = len(vocab)
    pairs: Counter[tuple[str, str]]
    while len_vocab < vocab_size:
        pairs = Counter()
        for symbols in symbols_list:
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i + 1])] += 1
        if not pairs:
            break
        bp_l, bp_r = max(pairs, key=lambda x: pairs[x])
        best_pair = bp_l + bp_r
        vocab.add(best_pair)
        len_vocab += 1
        # 更新文本中的该符号对
        new_symbols_list = []
        for symbols in symbols_list:
            new_symbols = []
            len_symbols = len(symbols) - 1
            i = 0
            while i < len_symbols:
                if (symbol := symbols[i]) == bp_l and symbols[i + 1] == bp_r:
                    new_symbols.append(best_pair)
                    i += 2
                else:
                    new_symbols.append(symbol)
                    i += 1
            else:
                if i == len_symbols:
                    new_symbols.append(symbols[i])
            new_symbols_list.append(new_symbols)
        symbols_list = new_symbols_list
    return vocab

In [4]:
import time

start = time.time()
vocab = train_bpe(train_data, 600)
end = time.time()
print(f"分词耗时：{end - start}")
print(vocab, len(vocab))

分词耗时：14.378931045532227
{'ill', 'Y', 'ul', 'ound', 'pl', 'sc', "I'", '$', 'tion', 'ganiz', '16', 'ence', 'sp', 'ain', 'ost', 'ell', 'P', 'ment', 'we', 'ber', 'cont', 'ast', 'Tur', 'ich', 'per', '(', 'ory', 'K', 'des', 'than', '====', 'Re:', 'w', 'pp', '!', 'when', 'not', 'ame', 'very', 'k', "'s", 'man', 'ers', 'ess', 'ri', '14', 'fore', 'lo', 'st', 'you', '!-*-', 'Z', 'ou', 'bu', '17', 'fin', 'ual', 'con', 'ont', 'r', 'only', 'es.', '19', 'them', 'ight', '25', 'ect', 'ks', 'T', 'ly', '__', 'ide', 'rom', 'ible', '%%%%', 'ans', 'Organization:', 'which', 'about', 'ld', 'y', '[', '========', '5', 'bl', 'ent', 'An', 'age', 'arm', 'op', 'ie', 'ro', 'ply', 'sy', 'ews', 'osting', 'az', '~', 'ec', 'x', 'ar', 'from', '--------------------------------', 'University', 'br', 'work', 'ause', 'am', '##', 'end', 'Armenian', 'In', 'X', 'ot', 'sa', 'ew', '20', '________', 'ition', 'vi', 'U', 's.', 'es:', 'ive', 'dri', 'C', 'ite', 'im', 'ong', 'af', 'will', 'may', ']', 'Lin', 'SCSI', 'ia', 'ce', 'icle', 

In [5]:
# 使用现有的第三方实现

from tokenizers import models, Tokenizer, trainers

tokenizer = Tokenizer(models.BPE())
start = time.time()
tokenizer.train_from_iterator(
    train_data,
    trainer=trainers.BpeTrainer(vocab_size=600),
)
std_vocab = set(tokenizer.get_vocab().keys())
end = time.time()
print(f"分词耗时：{end - start}")
print(std_vocab, len(std_vocab))

分词耗时：0.09062838554382324
{'ill', 'ound', 'ul', 'Y', 'pl', 'sc', 'ject:', "I'", '$', 'tion', '16', 'ence', 'sp', 'ain', 'ost', 'ell', 'P', 'ment', 'we', 'ber', 'cont', 'ast', 'Tur', 'ich', 'per', '(', 'ory', 'K', 'des', 'than', '====', 'Re:', 'w', 'pp', 'when', '!', 'ame', 'not', 'very', 'k', "'s", 'man', 'ers', 'ization:', 'ess', 'ri', '14', 'fore', 'lo', 'st', 'you', '!-*-', 'Z', 'ou', 'bu', '17', 'fin', 'ual', 'con', 'ont', 'r', 'only', 'es.', '19', 'them', '25', 'ight', 'ect', 'ks', 'ly', 'T', '__', 'ide', 'rom', 'ible', '%%%%', 'ans', 'which', 'Organization:', 'about', 'ld', 'y', '[', '========', '5', 'bl', 'An', 'ent', 'arm', 'age', 'op', 'ie', 'ro', 'ply', 'sy', 'ews', 'osting', 'az', '~', 'ec', 'x', 'ar', 'from', '--------------------------------', 'University', 'br', 'work', 'ause', 'am', '##', 'end', 'Armenian', 'In', 'X', 'ot', 'ew', '20', '________', 'ople', 'ition', 'vi', 'U', 's.', 'es:', 'ive', 'dri', 'C', 'ite', 'im', 'ong', 'Mar', 'af', 'will', 'may', ']', 'Lin', 'SCSI'

In [6]:
# 查看分词集合差异
print(vocab.difference(std_vocab))
print(std_vocab.difference(vocab))

{'peop', 'This', 'ation:', 'acc', 'sa', 'Organiz', 'tribution', '-Posting', 'ganiz', 'Subject'}
{'ople', 'ization:', '/9', 'Mar', 'ject:', 'Organ', 'ization', 'cess', 'mis', 'osting-H'}


可以发现我们自己的纯 Python 实现和 tokenizers 库的计算结果相差不大（600词汇相差10个词），可能是在统计最高频词时对同频次词汇处理方式不一致导致的差异。

但不管怎么说，tokenizers 库实现的速度要快的多。

# 基于深度学习的分词

基于深度学习的分词方法将分词任务建模为序列标注或序列生成问题，利用神经网络自动学习文本特征和分词规律。以下是其核心原理：

1. 问题建模方式

    (1) 序列标注方法（主流方法）
    
    将分词转化为字符级别的分类任务，常用标签体系：
    
    B/I/E/S：Begin(词首)/Inside(词中)/End(词尾)/Single(单字词)
    
    B/M/E/W/O：Begin/Middle/End/Whole(整词)/Outside(非词)
    ```
    示例：
    原句：深度学习很强大
    标注：B E B E S B E
    分词：深度/学习/很/强大
    ```

   (2) 序列生成方法
   
    将分词视为从字符序列到词序列的转换问题，使用Encoder-Decoder架构生成分词结果。

2. 关键技术特点

    上下文感知：
    
    能解决歧义切分问题 "下雨天留客天留我不留"
    
    自动识别未登录词

    端到端训练：
    
    无需人工设计特征
    
    自动学习有效的文本表示
    
    迁移学习：
    
    可利用预训练语言模型(如BERT)提升性能
    
    在小样本数据上表现良好

3. 优势与局限

    优势：
    
    自动学习复杂特征
    
    对未登录词自适应强
    
    多语言适应性强
    
    局限：
    
    需要大量标注数据
    
    模型可解释性差
    
    计算资源消耗大

*在本项目的代码库中实现了一个简单的使用深度学习训练的分词器，在此处不进行展开。*